In [113]:
import setup_runs as sr
import os, os.path
import pandas as pd
import numpy as np
import pyDOE as pyd
import time
import sys

#export files?
export_ed_files_q = True



##########################################
###                                    ###
###    GENERATE EXPERIMENTAL DESIGN    ###
###                                    ###
##########################################

print("Starting build of components for experimental design.")


#number of lhs samples
n_lhs = sr.dict_init["n_lhs"]
#set baseline strategy
strat_baseline = 0#[x for x in sr.dict_strat.keys() if sr.dict_strat[x]["strategy_id"] == 0][0]

###   SOME ATTRIBUTE TABLES
df_attribute_design_id = pd.read_csv(sr.fp_csv_attribute_design)
#reduce
df_attribute_design_id = df_attribute_design_id[df_attribute_design_id["include"] == 1]
#get time series id
df_attribute_time_series_id = pd.read_csv(sr.fp_csv_attribute_time_series)


print("Check: export_ed_files_q = " + str(export_ed_files_q))
print("Check: n_lhs = " + str(n_lhs))

    


#################################
#    MULTI SECTOR COMPONENTS    #
#################################


###   GET THE PARAMETER TABLE

#read in uncertainty table for additional sectors
parameter_table_additional_sectors = pd.read_csv(sr.fp_csv_parameter_ranges)
#reduce
parameter_table_additional_sectors = parameter_table_additional_sectors[parameter_table_additional_sectors["type"].isin(["incertidumbre", "accion"])]
#add field
parameter_table_additional_sectors["variable_name_lower"] = [x.lower().replace(" ", "_") for x in list(parameter_table_additional_sectors["parameter"])]
#fill nas and set to integer
parameter_table_additional_sectors["parameter_constant_q"] = parameter_table_additional_sectors["parameter_constant_q"].fillna(0)
parameter_table_additional_sectors["parameter_constant_q"] = np.array(parameter_table_additional_sectors["parameter_constant_q"]).astype(int)
#initialize available groups
groups_norm = set([int(x) for x in parameter_table_additional_sectors["normalize_group"] if not np.isnan(x)])

# IDENTIFY PARAMETERS THAT DO NOT VARY
df_apn = parameter_table_additional_sectors[["variable_name_lower", "min_2050", "max_2050"]].copy().drop_duplicates().reset_index(drop = True)
#initialize the set
all_params_novary = set({})
#loop to build
for p in df_apn["variable_name_lower"].unique():
    df_apn_tmp = df_apn[df_apn["variable_name_lower"] == p]
    #check to see if it doesn't vary
    if len(df_apn_tmp) == 1:
        if float(df_apn_tmp["min_2050"].iloc[0]) == float(df_apn_tmp["max_2050"].iloc[0]):
            if float(df_apn_tmp["min_2050"].iloc[0]) == 1.0:
                all_params_novary = all_params_novary | set({p})
#all parameters that vary
all_params_vary = set(parameter_table_additional_sectors["variable_name_lower"]) - all_params_novary




###   NORMALIZATION GROUP IDS

group_id = parameter_table_additional_sectors[["type", "parameter", "normalize_group"]].drop_duplicates()
#build normalize group and lever group ids (for deltas)
norm_vec = []

if len(groups_norm) > 0:
    #starting point for new group
    ind_group = max(groups_norm) + 1
else:
    ind_group = 1
    
#loop over rows
for i in range(0, len(group_id)):
    #get current group
    group_cur = group_id["normalize_group"].iloc[i]
    #test for NaN
    if np.isnan(group_cur):
        norm_vec = norm_vec + [ind_group]
        #next iteration
        ind_group += 1
    else:
        norm_vec = norm_vec + [int(group_cur)]

#add to group_id data frame
group_id["norm_group_id"] = norm_vec
group_id = group_id.reset_index(drop = True)


###   LEVER GROUP IDS

#temporary data frame to build lever group id with
group_id_tmp = group_id[["type", "norm_group_id"]].drop_duplicates()
group_id_tmp = group_id_tmp[group_id_tmp["type"].isin(["Accion", "accion"])]
group_id_tmp["lever_group_id"] = range(1, len(group_id_tmp) + 1)
if "lever_group_id" not in group_id.columns:
    group_id = pd.merge(group_id, group_id_tmp[["norm_group_id", "lever_group_id"]], how = "left", left_on = ["norm_group_id"], right_on = ["norm_group_id"])
#reduce
group_id = group_id[["parameter", "type", "norm_group_id", "lever_group_id"]]
#replace nas
group_id = group_id.fillna({"lever_group_id": -1})
#integer
group_id["lever_group_id"] = [int(x) for x in list(group_id["lever_group_id"])]
#merge in
group_id  = pd.merge(group_id, parameter_table_additional_sectors[["parameter", "variable_name_lower"]], how = "left", left_on = ["parameter"], right_on = ["parameter"])
group_id = group_id.drop_duplicates()
group_id = group_id[["parameter", "type", "variable_name_lower", "norm_group_id", "lever_group_id"]]



###   MERGE BACK IN TO PTAS AND CLEAN NAMES/DATA

#set merge fields and merge
fields_merge = list(set(parameter_table_additional_sectors.columns).intersection(set(group_id.columns)))
parameter_table_additional_sectors = pd.merge(parameter_table_additional_sectors, group_id, how = "left", left_on = fields_merge, right_on = fields_merge)
#fields to extract
fields_ext = [x for x in parameter_table_additional_sectors if (x not in ["normalize_group"])]
#clean the data frame
parameter_table_additional_sectors = parameter_table_additional_sectors[fields_ext]
#fields that are allowed to be all na
fields_allow_na = ["trajgroup_no_vary_q", "parameter_constant_q", "normalize_group"]
for field in fields_allow_na:
    if field in parameter_table_additional_sectors.columns:
        parameter_table_additional_sectors[field] = np.array(parameter_table_additional_sectors[field].fillna(0)).astype(int)
#drop nas
parameter_table_additional_sectors = parameter_table_additional_sectors.dropna(axis = 1, how = "all")

#dictionary to rename
dict_ptas_rename = dict([[x, x.lower().replace(" ", "_")] for x in parameter_table_additional_sectors.columns])
#set type
parameter_table_additional_sectors = parameter_table_additional_sectors.rename(columns = dict_ptas_rename)

#parameters to index
fields_add_sec_all_vals = ["strategy_id", "type", "parameter", "sector", "norm_group_id", "lever_group_id"]
#get parameter years that are defined
param_years_add_sec = [float(x) for x in parameter_table_additional_sectors.columns if x.replace(".", "").isnumeric()]
param_years_add_sec = [int(x) for x in param_years_add_sec if (int(x) == x)]

#initialize for all parameters
all_vals_add_sec = {
    "param_years": param_years_add_sec,
    "future_id": list(range(1, n_lhs + 1)),
    "design_id": list(df_attribute_design_id["design_id"]),
    "time_series_id": list(set(df_attribute_time_series_id["time_series_id"]) & set(parameter_table_additional_sectors["time_series_id"]))
}
#sort some
all_vals_add_sec["time_series_id"].sort()

#loop
for field in fields_add_sec_all_vals:
    #set the field name
    str_field = field.lower().replace(" ", "_")
    #
    if field in ["lever_group_id", "norm_group_id"]:
        set_field = set([x for x in parameter_table_additional_sectors[field] if x > 0])
    else:
        set_field = set(parameter_table_additional_sectors[field])
    #update the dictionary
    all_vals_add_sec.update({str_field: set_field})

    
    
    
###################################
#    GENERATE ATTRIBUTE TABLES    #
###################################

###   RUN ID
df_attribute_run_id_0 = [(x, y) for x in sr.df_strat["strategy_id"] for y in ([0] + all_vals_add_sec["future_id"])]
df_attribute_run_id = pd.DataFrame(df_attribute_run_id_0)
df_attribute_run_id = df_attribute_run_id.rename(columns = {0: "strategy_id", 1: "future_id"})
df_attribute_run_id["run_id"] = range(0, len(df_attribute_run_id))
df_attribute_run_id = df_attribute_run_id[["run_id", "strategy_id", "future_id"]]

if export_ed_files_q:
    #note/export
    print("Exporting run_id attribute to " + sr.fp_csv_attribute_runs)
    df_attribute_run_id.to_csv(sr.fp_csv_attribute_runs, index = None)

###   MASTER ID
df_attribute_master_id = [[x] + [y] + list(z) for x in all_vals_add_sec["design_id"] for y in all_vals_add_sec["time_series_id"] for z in df_attribute_run_id_0]
df_attribute_master_id = pd.DataFrame(df_attribute_master_id)
df_attribute_master_id = df_attribute_master_id.rename(columns = {0: "design_id", 1: "time_series_id", 2: "strategy_id", 3: "future_id"})
fields_df_attribute_master_id = list(df_attribute_master_id.columns)
df_attribute_master_id["master_id"] = list(range(0, len(df_attribute_master_id)))
df_attribute_master_id = df_attribute_master_id[["master_id"] + fields_df_attribute_master_id]
df_attribute_master_id = pd.merge(df_attribute_master_id, df_attribute_run_id, how = "left", on = ["strategy_id", "future_id"])
#fields to order by
fields_ord_dfm = ["master_id", "design_id", "time_series_id", "run_id", "strategy_id", "future_id"]
#reorder
df_attribute_master_id = df_attribute_master_id[fields_ord_dfm].sort_values(by = fields_ord_dfm)

if export_ed_files_q and not sr.tornado_q:
    #note/export
    print("Exporting master_id attribute to " + sr.fp_csv_attribute_master)
    #export
    df_attribute_master_id.to_csv(sr.fp_csv_attribute_master, index = None, encoding = "UTF-8")
    #export for gams
    df_attribute_master_id_gams = df_attribute_master_id[["master_id"]].copy().rename(columns = {"master_id": "Escenarios"})
    df_attribute_master_id_gams.to_csv(sr.fp_csv_gams_data_set_escenarios, index = None, encoding = "UTF-8")
    del df_attribute_master_id_gams

if sr.tornado_q:

    print("Building tornado design...\n")
    
    ####################################################
    ###                                              ###
    ###    BUILD TORNADO PLOT EXPERIMENTAL DESIGN    ###
    ###                                              ###
    ####################################################

    #id fields
    fields_id_ptas_internal = ["norm_group_id", "lever_group_id"]
    fields_id_ptas = [x for x in parameter_table_additional_sectors.columns if (x[-3:] == "_id") and (x not in fields_id_ptas_internal)]
    tuples_id_ptas = parameter_table_additional_sectors[fields_id_ptas].drop_duplicates()
    tuples_id_ptas = [tuple(x) for x in np.array(tuples_id_ptas)]

    field_param = "variable_name_lower"
    fields_param_years_add_sec = [str(x) for x in param_years_add_sec]

    #mix vector
    y_0 = int(sr.dict_init["add_sec_variation_start_year"]) - 1
    y_1 = max(param_years_add_sec)
    y_base = min(param_years_add_sec)
    vec_ramp_unc = sr.build_linear_mix_vec((y_0, y_1), (y_base, y_1))

    #initialize loop for master id
    m_id = 0
    f_id = 0
    #initialize attribute master id out
    df_master_out = []

    init_ed_q = True
    init_fut_q = True

    df_out = []
    #loop
    for ti in tuples_id_ptas:
        #copy the dataframe
        params_tmp = parameter_table_additional_sectors.copy()
        #reduce the dataframe t the approporiate subset
        for j in range(len(fields_id_ptas)):
            #get the field and reduce the dataframe
            field = fields_id_ptas[j]
            params_tmp = params_tmp[params_tmp[field] == ti[j]]
        params_tmp = params_tmp.sort_values(by = [field_param])
        #data frame to use in copying
        df_pt = params_tmp.copy().reset_index(drop = True)
        fields_new = list(df_pt[field_param])
        df_pt = df_pt[fields_param_years_add_sec].transpose()
        
        dict_conv = dict([x for x in zip(list(df_pt.columns), fields_new)])
        df_pt = df_pt.rename(columns = dict_conv).reset_index(drop = True)
        df_pt["year"] = [int(x) for x in fields_param_years_add_sec]
        
        #finally, commit changes to params_tmp
        params_tmp = params_tmp[params_tmp[field_param].isin(all_params_vary)]
        apv = list(params_tmp[field_param])
        
        ##  split out max and min
        df_pt_max = params_tmp[["sector", field_param, "parameter_constant_q", "max_2050"] + fields_param_years_add_sec].copy().rename(columns = {"max_2050": "range"})
        df_pt_min = params_tmp[["sector", field_param, "parameter_constant_q", "min_2050"] + fields_param_years_add_sec].copy().rename(columns = {"min_2050": "range"})
        
        #add fields
        df_pt_max["type"] = ["max" for x in range(len(df_pt_max))]
        df_pt_min["type"] = ["min" for x in range(len(df_pt_min))]
        
        #build parameters df and initialize futures
        params_tmp = pd.concat([df_pt_max, df_pt_min], axis = 0).sort_values(by = [field_param, "type"]).reset_index(drop = True)
        
        
        if init_fut_q:
            df_future_out = params_tmp[["sector", field_param, "type", "range", "parameter_constant_q"]].copy().reset_index(drop = True)
            df_future_out = df_future_out
            df_future_out["future_id"] = list(range(1, len(df_future_out) + 1))
            #build dict
            dict_fut = dict([[tuple(x[0:2]), int(x[2])] for x in np.array(df_future_out[[field_param, "type", "future_id"]])])
            #reorder
            df_future_out = df_future_out[["future_id", "sector", field_param, "type", "range", "parameter_constant_q"]].rename(columns = {field_param: "parameter", "range": "scale_value", "type": "range_value"})
            init_fut_q = False
            
        
        #array of baseline trajectories
        array_traj = np.array(params_tmp[fields_param_years_add_sec])
        #rows of this will be the ramp if the parameter is not constant, and 1 otherwise; apply to uncertinaty delta
        array_mix = np.array([int(x)*vec_ramp_unc + (1 - int(x))*np.ones(len(fields_param_years_add_sec)) for x in (params_tmp["parameter_constant_q"] == 0)])
        #ranges and scalar vector
        vec_ranges = np.array(params_tmp["range"])
        vec_scale = vec_ranges*np.array(params_tmp["2050"])
        vec_unc_delta = vec_scale - np.array(params_tmp["2050"])
        #baseline trajectory + the
        array_new = array_traj + (array_mix.transpose() * vec_unc_delta).transpose()
        if False:
            #rang values to use to overwrite scaled
            w_const = np.where(np.array(params_tmp["parameter_constant_q"]) == 1)[0].astype(int)
            #overwrite in the expansion vector
            v_r_inds = vec_ranges[w_const]
            np.put(vec_scale, w_const, v_r_inds)
            #np.put(vec_ranges, w_noconst, )
            #new array
            array_new = (vec_scale*array_mix.transpose()).transpose() + array_traj*(1 - array_mix)
            #i'm lazy, so loop to overwrite with parameter constant
            for ind in w_const:
                #number of columns
                m = array_new.shape[1]
                #index of positions to overwrite
                ind_ow = (ind*m + np.array(range(m))).astype(ind)
                #const mult vector
                vec_mult = array_traj[ind]*vec_ranges[ind]
                np.put(array_new, ind_ow, vec_mult)
        
        #intialize new design
        new_design = []
        
        #initialize fields for dfmid
        fields_dfmid = ["master_id"] + fields_id_ptas + ["future_id", "design_id"]
        #build vector
        vec_dfmid = np.array([m_id] + list(ti) + [0, 0]).astype(int)
        #repeat
        df_join = pd.DataFrame([vec_dfmid for i in range(len(df_pt))], columns = fields_dfmid)
        df_pt = pd.concat([df_join, df_pt], axis = 1)
        #add to output
        if init_ed_q:
            df_out.append(df_pt)
            init_ed_q = False
        else:
            df_out.append(df_pt[list(df_out[0].columns)])
            
        #add to attribute table
        df_master_out.append(vec_dfmid)
        
        m_id += 1
        
        for param in apv:
            for ty in params_tmp["type"].unique():
                df_pt_1 = df_pt[[x for x in df_pt.columns if (x[-3:] != "_id")]].copy()
                #get column
                ind_col = set(np.where(params_tmp[field_param] == param)[0]) & set(np.where(params_tmp["type"] == ty)[0])
                ind_col = list(ind_col)[0]
                #update with value from array
                df_pt_1[param] = array_new[ind_col]

                #get future id
                f_id = dict_fut[(param, ty)]
                vec_dfmid = np.array([m_id] + list(ti) + [f_id, 0]).astype(int)
                #repeat
                df_join = pd.DataFrame([vec_dfmid for i in range(len(df_pt))], columns = fields_dfmid)
                df_pt_1 = pd.concat([df_join, df_pt_1], axis = 1)
                df_out.append(df_pt_1[df_out[0].columns])
                #add to attribute table
                df_master_out.append(vec_dfmid)
                
                m_id += 1
    #initialize attribute for master id
    df_attribute_master_id = pd.DataFrame(df_master_out, columns = (["master_id"] + fields_id_ptas + ["future_id", "design_id"]))
    #build output experimental design
    df_out = pd.concat(df_out, axis = 0).reset_index(drop = True)
    
    #add a run id
    df_attribute_run_id = df_attribute_master_id[["strategy_id", "future_id"]].drop_duplicates()
    df_attribute_run_id["run_id"] = list(range(0, len(df_attribute_run_id)))
    #merge
    df_attribute_master_id = pd.merge(df_attribute_master_id, df_attribute_run_id, how = "left", on = ["strategy_id", "future_id"]).sort_values(by = ["master_id"]).reset_index(drop = True)
    df_out = pd.merge(df_out, df_attribute_run_id, how = "left", on = ["strategy_id", "future_id"]).sort_values(by = ["master_id", "year"]).reset_index(drop = True)
    
    #set some fields to sort
    fields_sort_tmp = [x for x in fields_dfmid if (x not in ["master_id"])] + ["run_id"]
    fields_sort_tmp.sort()
    fields_scen = ["master_id"] + fields_sort_tmp + ["year"]
    fields_dat = [x for x in df_out.columns if (x not in fields_scen)]
    fields_dat.sort()
    #reorder columns
    df_out = df_out[fields_scen + fields_dat]
    
    #loop to clean
    for field in fields_scen:
        df_out[field] = np.array(df_out[field]).astype(int)
    #clean master id
    df_attribute_master_id = df_attribute_master_id[["master_id"] + fields_sort_tmp]
    
    
    
    

    ######################################
    #    EXPORT FILES FOR TORNADO RUN    #
    ######################################
     
    ##  EXPORT MASTER ID ATTRIBUTE TABLES
    
    print("Exporting master_id attribute to " + sr.fp_csv_attribute_master)
    #export
    df_attribute_master_id.to_csv(sr.fp_csv_attribute_master, index = None, encoding = "UTF-8")
    df_attribute_run_id.to_csv(sr.fp_csv_attribute_runs, index = None, encoding = "UTF-8")
    #export for gams
    df_attribute_master_id_gams = df_attribute_master_id[["master_id"]].copy().rename(columns = {"master_id": "Escenarios"})
    df_attribute_master_id_gams.to_csv(sr.fp_csv_gams_data_set_escenarios, index = None, encoding = "UTF-8")
    del df_attribute_master_id_gams

    
    ##  EXPRT FUTURE ATTRIBUTE
    
    df_future_out.to_csv(sr.fp_csv_attribute_future, index = None, encoding = "UTF-8")

else:
    #############################
    #    GENERATE LHS MATRIX    #
    #############################

    print("Generating LHS Matrix...\n")

    #dimensions
    p_add_sec = len(all_vals_add_sec["parameter"])
    n_add_sec_levers = len(all_vals_add_sec["lever_group_id"])
    #generate latin hypercube sample
    matrix_lhs = pyd.lhs(p_add_sec + n_add_sec_levers, samples = n_lhs)
    #vector for delineating sub-matrices
    vec_submat_lhs = [0, p_add_sec, n_add_sec_levers]
    #set vector of submatrix index
    vec_submat_lhs_names = ["add_sec", "levers"]

    ##  set names for additional sectors
    dict_names_add_sec = [x.lower().replace(" ", "_") for x in all_vals_add_sec["parameter"]]
    dict_names_add_sec.sort()
    dict_names_add_sec = [[x, dict_names_add_sec[x]] for x in range(0, len(dict_names_add_sec))]
    dict_names_add_sec = dict(dict_names_add_sec)

    ##  set names for lever groups
    dict_names_levers = ["lever_group_" + str(x) for x in list(all_vals_add_sec["lever_group_id"])]
    dict_names_levers = [[x, dict_names_levers[x]] for x in range(0, len(dict_names_levers))]
    dict_names_levers = dict(dict_names_levers)

    #initialize the fields for data frames
    dict_submat_lhs_fields = {
        "add_sec": dict_names_add_sec,
        "levers": dict_names_levers
    }
    #setup the output file paths
    dict_submat_file_paths = {
        "add_sec": sr.fp_csv_lhs_table_multi_sector,
        "levers": sr.fp_csv_lhs_table_levers
    }

    #initialize
    dict_submat_lhs = {}

    if not sr.read_lhs_tables_q:
        print("\n#####\n#####    GENERATING LHS TABLES WITH " + str(n_lhs) + " SAMPLES\n#####\n")
        
        #break off components
        for i in range(0, len(vec_submat_lhs) - 1):
            #set the field name for the dictionary
            nm = vec_submat_lhs_names[i]
            #check
            if i > -1:
                #get the indeces
                p0 = sum(vec_submat_lhs[0:(i + 1)])
            else:
                p0 = 0
            #set upper limit
            p1 = sum(vec_submat_lhs[0:(i + 2)])
            #temporary dataframe
            df_tmp = pd.DataFrame(matrix_lhs[:, p0:p1], index = None)
            #add name field
            if dict_submat_lhs_fields[nm] != None:
                df_tmp = df_tmp.rename(columns = dict_submat_lhs_fields[nm])
                #set names
                nms = list(df_tmp.columns)
                #add run id
                df_tmp["future_id"] = all_vals_add_sec["future_id"]
                #reorder
                df_tmp = df_tmp[["future_id"] + nms]
            #update dictionary
            dict_submat_lhs.update({nm: df_tmp})

            #export raw lhs data
            if export_ed_files_q:
                #note/export
                print("Exporting LHS for " + nm + " to " + dict_submat_file_paths[nm])
                df_tmp.to_csv(dict_submat_file_paths[nm], index = None)
                
    else:
        #notify
        print("\n#####\n#####    READING IN LHS TABLES\n#####\n")
        #initialize list of "future ids"
        set_future_ids_read = set({})
        ##  initialie booleans
        
        #initialize the set of futures that are read
        init_sfir_q = True
        #default the imbalance query to fale
        set_imbalance_q = False
        #default exitting to false
        exit_q = False
        
        #Initialize
        dict_read_futures = {}
        #exit codes
        dict_exit_codes = {
            "set_imbalance": "Number of future_ids in LHS tables are not the same. Check the LHS files to ensure they are using the same future_id indexing.",
            "set_nomatch": "LHS Tables have future ids that do not match specificed number of lhs trials."
        }
        #initialize index
        i = 0
        #read in lhs tables
        while (i < len(vec_submat_lhs_names)) and not set_imbalance_q:
            #get current file
            nm = str(vec_submat_lhs_names[i])
            #get file path
            fp_read = dict_submat_file_paths[nm]
            #read it in
            df_tmp = pd.read_csv(fp_read)
            #reorder it
            #df_tmp = df_tmp[dict_submat_lhs_fields[nm]]
            #update dictionary
            dict_submat_lhs.update({nm: df_tmp})
            #check
            if init_sfir_q:
                #initialize
                set_future_ids_read = set(df_tmp["future_id"])
                #set of futures to compare to for individual exit
                set_future_ids_read_compare = set_future_ids_read
                #turn off initialization
                init_sfir_q = False
            else:
                #current set of Future IDs
                set_future_ids_read_cur = set(df_tmp["future_id"])
                #read in and update the set of intersectional futures
                set_future_ids_read = set_future_ids_read & set_future_ids_read_cur
                #check
                if set_future_ids_read_cur != set_future_ids_read_compare:
                    #if any set of futures doesn't match the first one, turn on the exit
                    sys.exit(dict_exit_codes["set_imbalance"])
            #notify of successful completion
            print(nm + " LHS table successfully read from " + fp_read)
            #next ieration
            i += 1

        #cut out 0 (some files may have it, some may not)
        set_future_ids_read = set_future_ids_read - set({0})
        #set of what should be the future ids
        set_check_future_ids = set(range(1, n_lhs + 1))
        #compare
        if set_future_ids_read != set_check_future_ids:
            sys.exit(dict_exit_codes["set_nomatch"])
            
    print("\nLHS complete.\n")


    ##  SET SOME NAMES
    fields_ed_add_sec = dict_submat_lhs["add_sec"].columns
    fields_ordered_parameters = [x for x in fields_ed_add_sec if x != "future_id"]
    sr.print_list_output(fields_ordered_parameters, "fields_ordered_parameters")

    #get parameters that are not ramped—they are constant across all years
    all_constant_params = set(parameter_table_additional_sectors[parameter_table_additional_sectors["parameter_constant_q"] == 1]["parameter"])
    all_constant_params = list(all_constant_params)
    all_constant_params.sort()

    sr.print_list_output(all_constant_params, "all_constant_params")

    #get indices
    indices_fop_all_constant_params = [fields_ordered_parameters.index(x) for x in all_constant_params]


    #################################################################
    ###                                                           ###
    ###    GENERATE EXPERIMENTAL DESIGN FOR ADDITIONAL SECTORS    ###
    ###                                                           ###
    #################################################################

    ###############################
    #    GENERATE LEVER DELTAS    #
    ###############################

    print("Starting generation of lever deltas.")
    print("")
    ##  START BUY BUILDING LONG TABLE OF TRANSFORMED LHS SAMPLES

    dsl = dict_submat_lhs["levers"].copy()
    dsl = pd.wide_to_long(dsl, i = ["future_id"], j = "lever_group_id", stubnames = "lever_group_")
    dsl = dsl.reset_index()
    dsl = dsl.rename(columns = {"lever_group_": "lhs_val"})
    #data frame out
    df_ld_lhs_transformed = []
    #loop over design id
    for did in all_vals_add_sec["design_id"]:
        #get applicable data
        dict_data = df_attribute_design_id[df_attribute_design_id["design_id"] == did].to_dict()
        #initialize
        lhs_out = dsl.copy()
        #get key for to_dict
        key = list(dict_data["vary_lever_deltas"].keys())[0]
        #get range of values
        vec_vals = np.array(lhs_out["lhs_val"])
        #set header for Future 0
        df_fut_0 = pd.DataFrame([[did, 0, x, 1] for x in all_vals_add_sec["lever_group_id"]], columns = ["design_id", "future_id", "lever_group_id", "lhs_val"])
        #check on varying
        if dict_data["vary_lever_deltas"][key] == 1:
            #transform
            m = float(dict_data["linear_transform_ld_m"][key])
            b = float(dict_data["linear_transform_ld_b"][key])
            #thresholds
            thresh_min = float(dict_data["min_lever_deltas"][key])
            thresh_max = float(dict_data["max_lever_deltas"][key])
            #transformation
            def linear_transform(x):
                return max(min(m*x + b, thresh_max), thresh_min)
            #updated vals
            vec_vals = list(map(linear_transform, vec_vals))
        else:
            #set the deltas to 1
            vec_vals = [1.0 for x in range(len(vec_vals))]
        #update in data frames
        lhs_out["lhs_val"] = vec_vals
        lhs_out["design_id"] = [did for x in range(len(vec_vals))]
        #update
        lhs_out = pd.concat([df_fut_0, lhs_out[["design_id", "future_id", "lever_group_id", "lhs_val"]]])
        #add
        df_ld_lhs_transformed = df_ld_lhs_transformed + [lhs_out]
    #convert to dataframee
    df_ld_lhs_transformed = pd.concat(df_ld_lhs_transformed)


    ##  THEN, GO BY STRATEGY TO BUILD DELTAS

    print("Starting build of deltas by time series/strategy...")
    print("")
    #set strategies to build deltas for
    strat_lever_deltas = [x for x in all_vals_add_sec["strategy_id"] if x != strat_baseline]
    #temporary reduction
    ptas_ld = parameter_table_additional_sectors[parameter_table_additional_sectors["lever_group_id"] > 0]

    #merge field for generate lever deltas
    fields_merge_ld = ["variable_name_lower", "lever_group_id"]
    #set extraction fields by type
    extraction_fields_ld = fields_merge_ld + [str(x) for x in param_years_add_sec]

    #dictionary of lever deltas by strategy
    dict_ld = {}
    #fields to extract and use in transpose
    fields_ext_ld = fields_ordered_parameters
    #initialize
    df_ld_shaped = []
    #loop over time series ids
    for ts_id in all_vals_add_sec["time_series_id"]:
        #get baseline data frame
        df_base = ptas_ld[(ptas_ld["strategy_id"] == strat_baseline) & (ptas_ld["time_series_id"] == ts_id)].copy()
        #reduce
        df_base = df_base[extraction_fields_ld]
        
        #loop over strategies to generate lever deltas by included years
        for strat in strat_lever_deltas:
            #get strategy id
            strat_id = int(strat)#int(sr.dict_strat_ids[strat])
            #get sub data frame
            df_strat = ptas_ld[(ptas_ld["strategy_id"] == strat) & (ptas_ld["time_series_id"] == ts_id)].copy()
            df_strat = df_strat[extraction_fields_ld]
            #data fields
            fields_data = [x for x in extraction_fields_ld if (x not in fields_merge_ld)]
            #get column rename
            dict_rename_df_strat = dict([[str(x), str(strat) + "_" + str(x)] for x in fields_data])
            #update
            df_strat = df_strat.rename(columns = dict_rename_df_strat)
            #merge in
            df_strat = pd.merge(df_strat, df_base, how = "inner", left_on = fields_merge_ld, right_on = fields_merge_ld)
            #set new lever delta fields
            fields_ld = []
            #generate subtraction
            for fd in fields_data:
                field_0 = fd
                field_s = str(strat) + "_" + str(fd)
                field_ld = "ld_" + field_s
                fields_ld = fields_ld + [field_ld]
                #parse out and get difference
                df_strat_tmp = df_strat[[field_0, field_s]].diff(axis = 1)
                #update
                df_strat[field_ld] = df_strat_tmp[field_s]
            #reduce
            df_strat = df_strat[fields_merge_ld + fields_ld]
            #merge in
            df_strat = pd.merge(df_strat, df_ld_lhs_transformed, how = "outer", left_on = ["lever_group_id"], right_on = ["lever_group_id"])
            #build new data
            df_deltas_adj = (np.array(df_strat[fields_ld]).transpose() * np.array(df_strat["lhs_val"])).transpose()
            df_deltas_adj = pd.DataFrame(df_deltas_adj, columns = fields_ld)
            #re-initialize
            df_strat = df_strat[["design_id", "future_id"] + fields_merge_ld]
            df_strat = pd.concat([df_strat, df_deltas_adj], axis = 1, sort = False)
            #update
            dict_ld.update({strat: df_strat})
            
            # BUILD RESHAPED VALUE
            
            #loop over design/future
            for did in list(df_strat["design_id"].unique()):
                for fut in list(df_strat["future_id"].unique()):
                    df_tmp = df_strat[(df_strat["design_id"] == did) & (df_strat["future_id"] == fut)].copy()
                    df_tmp = df_tmp[["variable_name_lower"] + fields_ld]
                    #fields to add
                    fields_new = list(df_tmp["variable_name_lower"])
                    #new data frame
                    df_tmp = pd.DataFrame(np.array(df_tmp[fields_ld]).transpose(), columns = fields_new)
                    #add year
                    df_tmp["year"] = param_years_add_sec
                    #add design id and future
                    df_tmp["design_id"] = [did for x in range(len(df_tmp))]
                    df_tmp["future_id"] = [fut for x in range(len(df_tmp))]
                    df_tmp["strategy_id"] = [strat_id for x in range(len(df_tmp))]
                    df_tmp["time_series_id"] = [ts_id for x in range(len(df_tmp))]
                    #order
                    df_tmp = df_tmp[["design_id", "time_series_id", "strategy_id", "future_id", "year"] + fields_new]
                    #update
                    df_ld_shaped = df_ld_shaped + [df_tmp]
                #notify of completed reshape
                print("Reshaping of LHS table complete for design_id: " + str(did) + ", time_series_id: " + str(ts_id) + ", strategy_id: " + str(strat))

    if len(df_ld_shaped) > 0:
        #convert to wide frame
        df_ld_shaped = pd.concat(df_ld_shaped)
        #rename
        df_ld_shaped = df_ld_shaped.rename(columns = dict([[x, "ld_" + x] for x in fields_new]))



Starting build of components for experimental design.
Check: export_ed_files_q = True
Check: n_lhs = 10
Exporting run_id attribute to /Users/jsyme/Documents/Projects/FY20/SWCHE102-1000/git/MultiSector_LTS_Chile/experimental_design/attribute_runs.csv
Exporting master_id attribute to /Users/jsyme/Documents/Projects/FY20/SWCHE102-1000/git/MultiSector_LTS_Chile/experimental_design/attribute_master.csv
Generating LHS Matrix...


#####
#####    GENERATING LHS TABLES WITH 10 SAMPLES
#####

Exporting LHS for add_sec to /Users/jsyme/Documents/Projects/FY20/SWCHE102-1000/git/MultiSector_LTS_Chile/experimental_design/lhs_samples_multi_sector.csv
Exporting LHS for levers to /Users/jsyme/Documents/Projects/FY20/SWCHE102-1000/git/MultiSector_LTS_Chile/experimental_design/lhs_samples_levers.csv

LHS complete.




##############################
fields_ordered_parameters:
	cement_emission_fact_coal
	cement_emission_fact_diesel
	cement_emission_fact_kerosene
	cement_emission_fact_natural_gas
	cement_fra

Reshaping of LHS table complete for design_id: 0, time_series_id: 0, strategy_id: 1
Reshaping of LHS table complete for design_id: 1, time_series_id: 0, strategy_id: 1
Reshaping of LHS table complete for design_id: 0, time_series_id: 1, strategy_id: 1
Reshaping of LHS table complete for design_id: 1, time_series_id: 1, strategy_id: 1


In [114]:
##  CREATE RAMP VECTORS FOR UNCERTAINTY

y_0 = int(sr.dict_init["add_sec_variation_start_year"]) - 1
y_1 = max(param_years_add_sec)
y_base = min(param_years_add_sec)
vec_ramp_unc = sr.build_linear_mix_vec((y_0, y_1), (y_base, y_1))
#sr.build_mix_vec(0, 0, 0, "linear")#np.array([max(min((y - y_0)/(y_1 - y_0), 1), 0) for y in param_years_add_sec])
vec_ramp_base = 1 - vec_ramp_unc


##  BUILD BASIC PERCENTAGE CHANGE MATRIX

dict_unc_delta = {}

In [115]:
#set lever delta parameters
ld_params = list(set(df_ld_shaped.columns) - set([x for x in df_ld_shaped.columns if (x[-3:] == "_id") or (x == "year")]))
ld_params = [x.replace("ld_", "") for x in ld_params]
ld_params.sort()


for ts_id in all_vals_add_sec["time_series_id"]:
    
    #set data frame of baseline percentage changes
    df_unc_delta = []
    
    for st_id in all_vals_add_sec["strategy_id"]:
        
        #assign baseline strategy to ld params
        df_ed_baseline = parameter_table_additional_sectors[(parameter_table_additional_sectors["time_series_id"] == ts_id)].copy()
        #(parameter_table_additional_sectors["strategy_id"] == strat_baseline) & 
        df_ed_baseline = df_ed_baseline[((df_ed_baseline["strategy_id"] == strat_baseline) & (df_ed_baseline["variable_name_lower"].isin(ld_params))) | ((df_ed_baseline["strategy_id"] == st_id) & (~df_ed_baseline["variable_name_lower"].isin(ld_params)))]
        #update strategy id to be uniform
        df_ed_baseline["strategy_id"] = np.array([st_id for x in range(len(df_ed_baseline))])
        #copy over
        df_merge_in = df_ed_baseline.copy()
        
        df_ed_base = df_ed_baseline[extraction_fields_ld].copy()
        df_ed_add_sec = dict_submat_lhs["add_sec"]

        #initialize array (in order of ext fields)
        array_ed_add_sec = np.array(df_ed_add_sec[fields_ordered_parameters])
        #order output array
        df_max_min = pd.merge(pd.DataFrame(fields_ordered_parameters, columns = ["variable_name_lower"]), df_ed_baseline[["variable_name_lower", "min_2050", "max_2050"]], how = "left", left_on = ["variable_name_lower"], right_on = ["variable_name_lower"])
        #generate transformed values of uncertainty
        array_ed_add_sec_trans = array_ed_add_sec * np.array(df_max_min["max_2050"]) + (1 - array_ed_add_sec) * np.array(df_max_min["min_2050"])
        #add in future 0 (0 change = 100%)
        array_ed_add_sec_trans = np.concatenate([np.ones((1, array_ed_add_sec_trans.shape[1])), array_ed_add_sec_trans])
        #get 2050 values for each parameter
        df_po = pd.DataFrame(fields_ordered_parameters, columns = ["variable_name_lower"])
        #df_merge_in = parameter_table_additional_sectors[(parameter_table_additional_sectors["strategy_id"] == strat_baseline) & (parameter_table_additional_sectors["time_series_id"] == ts_id)]
        df_po = pd.merge(df_po, df_merge_in[["variable_name_lower"] + [str(x) for x in param_years_add_sec]], how = "left", on = ["variable_name_lower"])
        vec_pvals_2050 = np.array(df_po["2050"])

        #build outcome matrix
        for i in range(len(param_years_add_sec)):
            y = param_years_add_sec[i]
            #build array of change from specified 2050 value
            array_tmp = (array_ed_add_sec_trans - 1)*vec_ramp_unc[i]
            #multiple everything by the delta from specified trajectory implied by the future
            array_tmp = array_tmp*vec_pvals_2050
            #remove the "ramp" component for constant params
            array_tmp[:, indices_fop_all_constant_params] = ((array_ed_add_sec_trans[:, indices_fop_all_constant_params] - 1).copy())*np.array(df_po[str(y)])[indices_fop_all_constant_params]
            #convert to data frame
            df_tmp = pd.DataFrame(array_tmp, columns = fields_ordered_parameters)
            #add year
            df_tmp["year"] = [int(y) for x in range(len(df_tmp))]
            df_tmp["future_id"] = [0] + list(df_ed_add_sec["future_id"])
            df_tmp["strategy_id"] = [st_id for x in range(len(df_tmp))]
            #organize
            df_tmp = df_tmp[["strategy_id", "future_id", "year"] + fields_ordered_parameters]
            #update
            df_unc_delta = df_unc_delta + [df_tmp]

    #build master
    df_unc_delta = pd.concat(df_unc_delta)
    df_unc_delta = df_unc_delta.sort_values(by = ["strategy_id", "future_id", "year"])

    #update dictionary
    dict_unc_delta.update({ts_id: df_unc_delta})
    

In [140]:
#dict_unc_delta[1][["strategy_id", "future_id", "manejo_holistico_de_gando"]]
#dict_unc_delta[1]["manejo_holistico_de_gando"]
#array_ed_add_sec_trans
#fields_ordered_parameters.index("manejo_holistico_de_gando")
#array_ed_add_sec_trans[:,262]

np.array(df_po[str(y)])[]



SyntaxError: invalid syntax (<ipython-input-140-5a08e55def11>, line 7)

In [54]:
##  BUILD DATA FRAME BASIS FOR EXPERIMENTAL DESIGN FILE

print("Building data frame basis for experimental design file...")
print("")

if len(df_ld_shaped) > 0:
    #fields that are in the LD matrix
    fields_ld_data = [x for x in df_ld_shaped.columns if (x.replace("ld_", "") in fields_ordered_parameters)]

    #print("\n"*8 + "#"*30)
    #print("\nSUCCESS\n\n")
    #print(df_ld_shaped)
    #print("\n\n" + "#"*30 + "\n"*8)
else:
    fields_ld_data = []
fields_ldparams_data = [x.replace("ld_", "") for x in fields_ld_data]
fields_nonldparams_data = [x for x in fields_ordered_parameters if (x not in fields_ldparams_data)]
#initialize output data frame
df_out = []

Building data frame basis for experimental design file...



In [55]:
ts_id = 0
did = 0

#get baseline for each time series
df_ed_baseline = parameter_table_additional_sectors[(parameter_table_additional_sectors["strategy_id"] == strat_baseline) & (parameter_table_additional_sectors["time_series_id"] == ts_id)].copy()
df_ed_base = df_ed_baseline[extraction_fields_ld].copy()
#set column headers
fields_df_ed = list(df_ed_base["variable_name_lower"])
df_ed_base = df_ed_base.transpose().loc[[str(x) for x in param_years_add_sec],:]
df_ed_base = df_ed_base.rename(columns = dict([[list(df_ed_base.columns)[x], fields_df_ed[x]] for x in range(len(fields_df_ed))]))
df_ed_base["year"] = [int(x) for x in df_ed_base.index]

In [56]:
#get applicable data
dict_data = df_attribute_design_id[df_attribute_design_id["design_id"] == did].to_dict()
#get key for to_dict
key = list(dict_data["vary_lever_deltas"].keys())[0]
#experimental design merge table
df_ed_merge = df_attribute_master_id[(df_attribute_master_id["time_series_id"] == ts_id) & (df_attribute_master_id["design_id"] == did)]
list_ed_merge = []
cols = [x for x in df_ed_merge.columns] + ["year"]
for y in param_years_add_sec:
    df_tmp = df_ed_merge.copy()
    df_tmp["year"] = [int(y) for x in range(len(df_tmp))]
    df_tmp = df_tmp[cols]
    list_ed_merge = list_ed_merge + [df_tmp]
#convert
df_ed_merge = pd.concat(list_ed_merge)
#add in
df_ed_merge = pd.merge(df_ed_merge, df_ed_base, how = "outer", left_on = ["year"], right_on = ["year"])
#renaming dictionaryw
dict_rnm = dict([x, "unc_delta_" + x] for x in fields_ordered_parameters)
#breakout fields
fields_unc_delta = [dict_rnm[x] for x in fields_ordered_parameters]

In [57]:
dict_data["vary_uncertainties"][key] == 1

True

In [58]:
df_ed_merge[["strategy_id", "future_id", "design_id", "time_series_id", "year"]]

,master_id,design_id,time_series_id,run_id,strategy_id,future_id,year,electrolyzer_efficiency,share_electric_grid_to_hydrogen,manejo_holistico_de_gando,...,transport_share_diesel_private,transport_share_diesel_truck,transport_tkm_freight_train,transport_tkm_maritime,transport_train13_pkm,transport_train5_pkm,transport_train8_pkm,transport_trip_distance_private,transport_truck_investment_cost_diesel,transport_truck_investment_cost_hydrogen
0,0,0,0,0,0,0,2015,0.73,1,0,...,0.32,0.68,4.08572e+09,1.62697e+07,6.0242e+09,3.73455e+08,2.1095e+07,7,71301.9,148892
1,1,0,0,1,0,1,2015,0.73,1,0,...,0.32,0.68,4.08572e+09,1.62697e+07,6.0242e+09,3.73455e+08,2.1095e+07,7,71301.9,148892
2,2,0,0,2,0,2,2015,0.73,1,0,...,0.32,0.68,4.08572e+09,1.62697e+07,6.0242e+09,3.73455e+08,2.1095e+07,7,71301.9,148892
3,3,0,0,3,0,3,2015,0.73,1,0,...,0.32,0.68,4.08572e+09,1.62697e+07,6.0242e+09,3.73455e+08,2.1095e+07,7,71301.9,148892
4,4,0,0,4,0,4,2015,0.73,1,0,...,0.32,0.68,4.08572e+09,1.62697e+07,6.0242e+09,3.73455e+08,2.1095e+07,7,71301.9,148892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
787,17,0,0,17,1,6,2050,0.73,1,0,...,0.32,0.68,3.86192e+09,2.73593e+07,1.03217e+10,1.92881e+09,8.65374e+07,7,71301.9,81438.7
788,18,0,0,18,1,7,2050,0.73,1,0,...,0.32,0.68,3.86192e+09,2.73593e+07,1.03217e+10,1.92881e+09,8.65374e+07,7,71301.9,81438.7
789,19,0,0,19,1,8,2050,0.73,1,0,...,0.32,0.68,3.86192e+09,2.73593e+07,1.03217e+10,1.92881e+09,8.65374e+07,7,71301.9,81438.7
790,20,0,0,20,1,9,2050,0.73,1,0,...,0.32,0.68,3.86192e+09,2.73593e+07,1.03217e+10,1.92881e+09,8.65374e+07,7,71301.9,81438.7


In [89]:
#loop over time series
#for ts_id in all_vals_add_sec["time_series_id"]:
#    for st_id in all_vals_add_sec["strategy_id"]:
ts_id = 0
st_id = 1
#assign baseline strategy to ld params and current strategy to others
df_ed_baseline = parameter_table_additional_sectors[(parameter_table_additional_sectors["time_series_id"] == ts_id)].copy()
df_ed_baseline = df_ed_baseline[((df_ed_baseline["strategy_id"] == strat_baseline) & (df_ed_baseline["variable_name_lower"].isin(ld_params))) | ((df_ed_baseline["strategy_id"] == st_id) & (~df_ed_baseline["variable_name_lower"].isin(ld_params)))]
df_ed_base = df_ed_baseline[extraction_fields_ld].copy()
#set column headers
fields_df_ed = list(df_ed_base["variable_name_lower"])
df_ed_base = df_ed_base.transpose().loc[[str(x) for x in param_years_add_sec],:]
df_ed_base = df_ed_base.rename(columns = dict([[list(df_ed_base.columns)[x], fields_df_ed[x]] for x in range(len(fields_df_ed))]))
df_ed_base["year"] = [int(x) for x in df_ed_base.index]


In [92]:
did = 1
#loop over design ids
#for did in all_vals_add_sec["design_id"]:
#get applicable data
dict_data = df_attribute_design_id[df_attribute_design_id["design_id"] == did].to_dict()
#get key for to_dict
key = list(dict_data["vary_lever_deltas"].keys())[0]
#experimental design merge table
df_ed_merge = df_attribute_master_id[(df_attribute_master_id["time_series_id"] == ts_id) & (df_attribute_master_id["strategy_id"] == st_id) & (df_attribute_master_id["design_id"] == did)]
list_ed_merge = []
cols = [x for x in df_ed_merge.columns] + ["year"]
for y in param_years_add_sec:
    df_tmp = df_ed_merge.copy()
    df_tmp["year"] = [int(y) for x in range(len(df_tmp))]
    df_tmp = df_tmp[cols]
    list_ed_merge = list_ed_merge + [df_tmp]
#convert
df_ed_merge = pd.concat(list_ed_merge)
#add in
df_ed_merge = pd.merge(df_ed_merge, df_ed_base, how = "outer", left_on = ["year"], right_on = ["year"])
#renaming dictionary
dict_rnm = dict([x, "unc_delta_" + x] for x in fields_ordered_parameters)
#breakout fields
fields_unc_delta = [dict_rnm[x] for x in fields_ordered_parameters]

#check if percentages need to be accounted for
if dict_data["vary_uncertainties"][key] == 1:
    #get the total
    df_tmp = dict_unc_delta[ts_id].copy()
    #rename to columns
    df_tmp = df_tmp.rename(columns = dict_rnm)
    df_ed_merge = pd.merge(df_ed_merge, df_tmp, how = "left", on = ["future_id", "strategy_id", "year"])
    #id fields to breakout on
    fields_id = [x for x in df_ed_merge.columns if (x not in fields_unc_delta) and (x not in fields_ordered_parameters)]
    #seprate out
    df_ed_merge_ids = df_ed_merge[fields_id]

    #lcheckr = list(df_ed_merge.columns)
    #checkr = [x for x in lcheckr if (lcheckr.count(x) > 1)]
    #sr.print_list_output(list(checkr), "checkr")
    #print("Length df_ed_merge cols: " + str(len(df_ed_merge.columns)))
    #print("Length df_ed_merge unique cols: " + str(len(set(df_ed_merge.columns))))

    #convert to sum of two arrays—has headers fields_ordered_parameters
    array_design = np.array(df_ed_merge[fields_ordered_parameters]) + np.array(df_ed_merge[fields_unc_delta])
else:
    #id fields to breakout on
    fields_id = [x for x in df_ed_merge.columns if (x not in fields_ordered_parameters)]
    #seprate out
    df_ed_merge_ids = df_ed_merge[fields_id]
    #reduce the design
    array_design = np.array(df_ed_merge[fields_ordered_parameters])

#update
df_ed_merge = pd.concat([df_ed_merge_ids, pd.DataFrame(array_design, columns = fields_ordered_parameters)], axis = 1, sort = False)

#check if lever delta needs to be brought in
#if dict_data["vary_lever_deltas"][key] == 1:

# ADD IN LEVER DELTAS

if len(df_ld_shaped) > 0:
    #fields to merge on
    fields_merge_ld = list(set(df_ed_merge.columns) & set(df_ld_shaped.columns))
else:
    fields_merge_ld = []




In [ ]:
if len(fields_merge_ld) > 0:
    #merge in lever deltas
    df_ed_merge = pd.merge(df_ed_merge, df_ld_shaped, how = "left", left_on = fields_merge_ld, right_on = fields_merge_ld)



In [110]:
#split out
df_ed_merge_ids = df_ed_merge[fields_id + fields_nonldparams_data]
#add in lever deltas
array_design = np.array(df_ed_merge[fields_ldparams_data]) + np.array(np.array(df_ed_merge[fields_ld_data].fillna(0)))

#reduce

In [112]:
pd.DataFrame(array_design, columns = fields_ldparams_data)

,manejo_holistico_de_gando,medida_cambio_dieta_nacional,medida_captura_c_suelos,medida_algas_pardas,medida_aumento_parques_reservas,medida_forestacion_aumentada,medida_manejo_aumentada,medida_pmr_menos_papel,nueva_cap_recuperacion,fp_compost,...,trajgroup_8-trajmax_steel_heat_biomass,trajgroup_8-trajmax_steel_heat_coal,trajgroup_8-trajmax_steel_heat_coke,trajgroup_8-trajmax_steel_heat_hydrogen,trajmin_residential_retrofit_house,trajmix_residential_retrofit_house,trajgroup_9-trajmax_transport_frac_aviation_hydrogen,trajgroup_9-trajmax_transport_frac_aviation_kerosene,trajgroup_9-trajmix_transport_frac_aviation_hydrogen,trajgroup_9-trajmix_transport_frac_aviation_kerosene
0,1,1,1,0,0,0,0,0,0,1,...,0,0.590901,0.399395,0,0,0,0,1,1,1
1,1,1,1,0,0,0,0,0,0,1,...,0,0.590901,0.399395,0,0,0,0,1,1,0.694657
2,1,1,0.765096,0,0,0,0,0,0,1,...,0,0.590901,0.399395,0,0,0,0,1,0.982466,0.899876
3,0.7511,0.767313,0.655397,0,0,0,0,0,0,0.602949,...,0,0.590901,0.399395,0,0,0,0,1,1,1
4,0.507602,0.696099,0.810456,0,0,0,0,0,0,0.532332,...,0,0.590901,0.399395,0,0,0,0,1,0.789991,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,1,0.943214,1,0.755493,1,0.547997,1,1,2800,1,...,0.0997281,0.491172,0.299667,0.0729772,3.71368e+06,0.539385,0.1,0.9,1,1
392,1,1,0.981111,1,1,0.612125,0.866683,0.55744,2800,0.928024,...,0.0997281,0.523974,0.299667,0.0608782,6.19775e+06,0.873215,0.0521373,0.9449,1,0.56223
393,0.982956,0.810119,0.529844,0.881101,1,0.809836,1,0.908954,1778.16,0.774722,...,0.0901013,0.491172,0.299667,0.0800808,5.79116e+06,0.233623,0.0679113,0.9,0.804271,1
394,0.88046,0.554644,1,0.961946,1,1,0.614417,1,2800,1,...,0.0766702,0.499474,0.312275,0.0997281,4.57695e+06,0.371111,0.0972697,0.9,1,0.973577


In [ ]:
#concatenate
df_ed_merge = pd.concat([df_ed_merge_ids, pd.DataFrame(array_design, columns = fields_ldparams_data)], axis = 1, sort = False)


In [ ]:

df_ed_merge = df_ed_merge[df_ed_merge["strategy_id"].isin(all_vals_add_sec["strategy_id"])]
#sort
df_ed_merge = df_ed_merge.sort_values(by = ["master_id", "year"])
#clear index
df_ed_merge = df_ed_merge.reset_index(drop = True)
#order
df_ed_merge = df_ed_merge[fields_id + fields_ordered_parameters]

if len(df_out) == 0:
    #add to output list
    df_out = df_out + [df_ed_merge]
else:
    df_out = df_out + [df_ed_merge[df_out[0].columns]]

print("Data frame for time_series_id: " + str(ts_id) + ", strategy_id: " + str(st_id) + ", design_id: " + str(did) + " complete.")
print("")

In [81]:
df_out = pd.concat(df_out, axis = 0)
#set ordering
fields_id = [x for x in df_out.columns if ("_id" == x[-3:])]
fields_dat = [x for x in df_out.columns if (x not in (fields_id + ["year"]))]
df_out = df_out[fields_id + ["year"] + fields_dat]

In [88]:
fields = ["trajgroup_8-trajmax_steel_heat_coke"]

df_out[df_out["year"].isin([2050]) & df_out["future_id"].isin([0, 1])][["time_series_id", "strategy_id", "future_id", "design_id"] + fields]



,time_series_id,strategy_id,future_id,design_id,trajgroup_8-trajmax_steel_heat_coke
35,0,0,0,0,0.399395
71,0,0,1,0,0.399395
35,0,0,0,1,0.399395
71,0,0,1,1,0.399395
35,0,1,0,0,0.299667
71,0,1,1,0,0.299667
35,0,1,0,1,0.299667
71,0,1,1,1,0.322974
35,1,0,0,0,0.399395
71,1,0,1,0,0.399395
